In [1]:
# https://github.com/davecom/ClassicComputerScienceProblemsInPython/blob/master/Chapter2/maze.py

### Project Description

Build a maze solving algorithm
- Build a maze
- Solve the maze

In [112]:
import random
from enum import Enum
from typing import List, NamedTuple, Generic, T, Optional

In [102]:
class Cell(str, Enum):
    """A class to represent the value of individual maze location.
    """
    EMPTY = "."
    BLOCKED = "X"
    PATH = "*"
    GOAL = "G"
    START = "S"

In [103]:
class MazeLocation(NamedTuple):
    """A class to represent individual maze location.
    """
    row: int
    column: int

In [104]:
class Maze:
    """A class to represent overall maze.
    """
    def __init__(self, rows: int = 10, columns: int = 10, sparseness: float = 0.2,
                start: MazeLocation = MazeLocation(0,0), goal: MazeLocation = MazeLocation(9,9)) -> None:
        self._rows: int = rows
        self._columns: int = columns
        self._sparseness = sparseness
        self.start: MazeLocation = start
        self.goal: MazeLocation = goal
        # fill the grid with empty cells
        self._grid: List[List[Cell]] = [
            [random.choices([Cell.EMPTY, Cell.BLOCKED], weights=(80, 20), k=1)[0] for c in range(self._columns)] 
                for r in range(self._rows)
        ]
        self._grid[self.start.row][self.start.column] = Cell.START
        self._grid[self.goal.row][self.goal.column] = Cell.GOAL
        
    def goal_test(self, ml: MazeLocation) -> bool:
        return ml == self.goal
    
    def sucessors(self, ml: MazeLocation) -> List[MazeLocation]:
        locations: List[MazeLocation] = []
        if ml.column-1 >= 0 and self._grid[ml.row][ml.column-1] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row, ml.column-1))
        if ml.column+1 < self._columns and self._grid[ml.row][ml.column+1] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row, ml.column+1))
        if ml.row-1 >= 0 and self._grid[ml.row-1][ml.column] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row-1, ml.column))  
        if ml.row+1 < self._rows and self._grid[ml.row+1][ml.column] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row+1, ml.column))
        
        return locations
    
    def __str__(self) -> str:
        board_output = ""
        for row in range(self._rows):
            for column in range(self._columns):
                board_output += self._grid[row][column]
            board_output += "\n"
        return board_output

In [105]:
m = Maze()
print(m)

S......XX.
..X.....X.
..........
..XX..X...
X...X.....
...X.XX.X.
.....XXX..
.X........
..XX......
XX.XX....G



In [106]:
print(m.sucessors(MazeLocation(9,0)))

[MazeLocation(row=8, column=0)]


In [107]:
class Stack(Generic[T]):
    def __init__(self) -> None:
        self._containsers: List[T] = []
    
    @property
    def empty(self) -> bool:
        return len(self._containers) == 0
    
    def push(self, item: T) -> None:
        self._containers.append(item)
    
    def pop(self) -> T:
        return self._containsers.pop()
    
    def __repr__(self) -> str:
        return repr(self._containers)

In [115]:
class Node(Generic[T]):
    def __init__(self, state: T, parent, cost:float=0.0,
                heuristic: float=0.0) -> None:
        self.state: T = state
        self.parent = parent
        self.cost: float = cost
        self.heuristic: float = heuristic
    
    def __lt__(self, other) -> bool:
        return (self.cost + self.heuristic) < (other.cost + other.heuristic)
    
    

In [116]:
def dfs(initial, goal_test, successors):
    # frontier is where we've yet to go
    frontier = Stack()
    frontier.push(Node(initial, None))
    # explored is where we've been
    explored = (initial)
    
    while not frontier.empty:
        current_node = frontier.pop()
        current_state = current_node.state
        # if we found the goal, we're done
        if goal_test(current_state):
            return current_node
        
        for child in successors(current_state):
            if child in explored:
                continue
            explored.add(child)
            frontier.push(Node(child, current_node))
    return None
    
    